In [81]:
# Необходимые библиотеки

!pip3 -qq install nltk --progress-bar off
!pip3 -qq install rusenttokenize --progress-bar off
!pip3 -qq install pymorphy2 --progress-bar off
!pip3 install -U scikit-learn --progress-bar off
from nltk.corpus import stopwords
from nltk import word_tokenize
from rusenttokenize import ru_sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import codecs 
import re
import nltk
import pymorphy2
import string
import random
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data]     C:\Users\79608\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\79608\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

You should consider upgrading via the 'C:\Users\79608\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


True

In [163]:
# Структура чат-бота

print('Привет! Я бот, который отвечает афоризмами великих мыслителей.\nО чём ты хочешь порассуждать? Сформулируй запрос, и я подберу для тебя афоризм :)\n\nПримеры запросов:\n\nЧто такое добро?\nПрошлое и будущее\nО суете житейской', flush=True)
while True: 
    question = input()
    if get_intent(question) != 'bye':
        answer = generate_response(question)
        print(answer)
        if answer in failure_phrase:
            continue
    else:
        print(random.choice(bye_responses))
        break
    print('Хочешь посмотреть ещё одно высказывание на эту тему?', flush=True)
    intention = input()
    if get_intent(question) != 'bye':
        answer = bot(intention)
        if answer in yes_responses:
            answer = generate_response_again(question)
            print(answer,'\nПожалуйста, сформулируй новый запрос.')
        elif answer in no_responses:
            print(random.choice(no_responses))
    else:
        print(random.choice(bye_responses))
        break       

Привет! Я бот, который отвечает афоризмами великих мыслителей.
О чём ты хочешь порассуждать? Сформулируй запрос, и я подберу для тебя афоризм :)

Примеры запросов:

Что такое добро?
Прошлое и будущее
О суете житейской
Что такое истинная красота?
Отлично! Посмотрим, что я нашёл: Истинная красота и мудрость всегда в простоте.
Хочешь посмотреть ещё одно высказывание на эту тему?
хочу
Истинные удовольствия невозможны без истинных потребностей. 
Пожалуйста, сформулируй новый запрос.
Пока!
Приходи ещё!


In [165]:
# Функция, возвращающая либо релевантный отклик, либо заглушку

def bot(question):
    intent = get_intent(question)
    if intent:
        answer = get_answer_by_intent(intent)
        if answer:
            return answer   
    answer = get_failure_phrase()
    return answer

In [170]:
# Словарь интентов с примерами реплик и откликами на них

BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': ['Привет', 'Добрый день', 'Здравствуйте'],
            'responses': ['Привет-привет!', 'День добрый!', 'Приветствую!']
        }, 
        'bye': {
            'examples': ['Пока', 'До свидания', 'Счастливо', 'Всего доброго', 'Всего хорошего', 'До скорого', 'Увидимся'],
            'responses': ['Ещё увидимся!', 'Если что, я тут', 'Приходи ещё!']
        },
        'yes': {
            'examples': ['Да', 'Конечно', 'Давай', 'Хочу', 'Посмотрю', 'Согласен', 'С радостью', 'Очень', 'Можно', 'Хорошо', 'Буду рад'],
            'responses': ['Хорошо, сейчас подберу ещё одно высказывание.', 'Хорошо, давай посмотрим.', 'Понял тебя. Немного подожди.']
        },
        'no': {
            'examples': ['Нет', 'Не нужно', 'Не надо', 'Хочу другое', 'Хочу на другую тему', 'На другую', 'Другой афоризм'],
            'responses': ['Понял тебя! Введи новый запрос, и я подыщу другой афоризм.\nЕсли захочешь выйти, просто попрощайся :)', 'Хорошо, поищем другой афоризм. Пожалуйста, сформулируй новый запрос.\nЕсли захочешь выйти, просто попрощайся :)', 'Я понял. Буду искать другой афоризм. Введи новый запрос.\nЕсли захочешь выйти, просто попрощайся :)']
        } 
        
    },
    "failure_phrases": [
        'Пожалуйста, переформулируйте мысль',
        'Попробуйте ещё раз', 'Не совсем вас понял'
    ]
}

In [74]:
# Фильтрует пользовательский ввод

def filter_text(text):
    text = text.lower()
    text = [c for c in text if c in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя- ']
    text = ''.join(text)
    return text

# Возвращает интент с учётом исправления опечаток

def get_intent(question):
    for intent, intent_data in BOT_CONFIG['intents'].items():
        for example in intent_data['examples']:
            dist = nltk.edit_distance(filter_text(example), filter_text(question))
            if dist / len(filter_text(example)) < 0.4:
                return intent

In [76]:
# Возвращает рандомный отклик в пределах конкретного интента

def get_answer_by_intent(intent):
    if intent in BOT_CONFIG['intents']:
        phrases = BOT_CONFIG['intents'][intent]['responses']
        return random.choice(phrases)

In [75]:
# Возвращает ответ на неудачный запрос

def get_failure_phrase():
    phrases = BOT_CONFIG['failure_phrases']
    return random.choice(phrases)

In [88]:
# Адреса примеров и откликов для интентов

bye_examples = BOT_CONFIG['intents']['bye']['examples']
bye_responses = BOT_CONFIG['intents']['bye']['responses']
yes_examples = BOT_CONFIG['intents']['yes']['examples']
yes_responses = BOT_CONFIG['intents']['yes']['responses']
no_examples = BOT_CONFIG['intents']['no']['examples']
no_responses = BOT_CONFIG['intents']['no']['responses']
failure_phrase = BOT_CONFIG['failure_phrases']

In [85]:
# Чтение файла "Всемирная энциклопедия афоризмов" Елены Агеевой

with codecs.open('Ageeva_clean.txt', 'r', "utf_8_sig") as file:
    aphorisms = file.read()
# display(aphorisms)    

In [84]:
# Очистка афоризмов от лишних символов и переносов

aphorisms_clean = re.sub('\xa0|[\r\n]+|//', ' ', aphorisms)
# display(aphorisms_clean)

# with codecs.open('Ageeva_cleaned.txt', 'w', "utf_8_sig") as file_cleaned:
#     file_cleaned.write(aphorisms_clean)

In [87]:
# Токенизация афоризмов на предложения 

aphorisms_sentences = ru_sent_tokenize(aphorisms_clean)
# display(aphorisms_sentences)
# aphorisms_words = nltk.word_tokenize(aphorisms_clean)
# display(aphorisms_words)

In [168]:
# Добавление некоторых слов в stopwords

morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')
russian_stopwords.extend(['почему', 'отчего', 'для чего', 'такое', 'такая', 'откуда'])
# print(russian_stopwords)

In [140]:
# Предобработка текста

def preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        
        clean_words = [word.lower() for word in clean_words if word not in russian_stopwords]
        
        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
        
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized

In [90]:
# Функция возвращает наиболее похожий на пользовательский запрос афоризм 

def generate_response(user_input):
    aphorisms_sentences.append(user_input)
    bot_response = ""
    
    word_vectorizer = TfidfVectorizer(tokenizer=preprocess_tokenize)
    all_words_vectors = word_vectorizer.fit_transform(aphorisms_sentences)
    similar_vector_values = cosine_similarity(all_words_vectors[-1], all_words_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]
    del aphorisms_sentences[-1]

    if vector_matched == 0:
        bot_response = bot_response + get_failure_phrase()
        return bot_response
    elif 0 < vector_matched <= 0.5:
        bot_response = bot_response + random.choice(small_similarity) + '\033[1m' + aphorisms_sentences[similar_sentence_number] + '\033[0m'
        return bot_response
    else:
        bot_response = bot_response + random.choice(big_similarity) + '\033[1m' + aphorisms_sentences[similar_sentence_number] + '\033[0m'
        return bot_response 

In [91]:
# Возвращает второй по похожести афоризм 

def generate_response_again(user_input):
    aphorisms_sentences.append(user_input)
    bot_response = ""
    
    word_vectorizer = TfidfVectorizer(tokenizer=preprocess_tokenize)
    all_words_vectors = word_vectorizer.fit_transform(aphorisms_sentences)
    similar_vector_values = cosine_similarity(all_words_vectors[-1], all_words_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-3]
    
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]
    del aphorisms_sentences[-1]

    if vector_matched == 0:
        bot_response = bot_response + get_failure_phrase()
        return bot_response
    else:
        bot_response = bot_response + '\033[1m' + aphorisms_sentences[similar_sentence_number] + '\033[0m'
        return bot_response

In [151]:
# Имитация общения

small_similarity = ['Кажется, я тебя понял.\nНа этот счёт мудрые говорят: ', 
                    'Хм! Возможно, кое-что подойдет.\n', 
                    'Вот что мне удалось найти: ']
big_similarity = ['Отлично! Посмотрим, что я нашёл: ',
                  'Превосходно! По этому поводу один философ сказал: ',
                  'Великие умы рассуждают так: ',
                  'Замечательно. Умудрённые жизнью говорят так: ',
                  'Я понял! У меня есть для тебя кое-что: ']        